In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from lenet import Lenet
from dataset import chunks
from datetime import datetime
import convolution as c
import numpy as np

iterations = 4000
batch_size = 50
display_step = 100 # summary save step
show_step = 10*display_step # print accuracy step
lr = 1e-4

mnist = input_data.read_data_sets('/tmp/dataset', one_hot=True)
test = list(chunks(mnist.test.images, 50)), list(chunks(mnist.test.labels, 50))
path = '/tmp/mnist/'
writer = tf.summary.FileWriter(path)

Extracting /tmp/dataset/train-images-idx3-ubyte.gz
Extracting /tmp/dataset/train-labels-idx1-ubyte.gz
Extracting /tmp/dataset/t10k-images-idx3-ubyte.gz
Extracting /tmp/dataset/t10k-labels-idx1-ubyte.gz


In [2]:
def train(sess, data, train_step, batch, prob, lr):
    train_step.run(session=sess, feed_dict={data[0]: batch[0], 
                                            data[1]: batch[1], 
                                            data[2]: prob, 
                                            data[3]: lr})
    
def show_acc(sess, data, accuracy, iteration):
    acc = []
    for i in range(len(test[0])):
        acc.append(accuracy.eval(session=sess, feed_dict={data[0]: test[0][i],
                                                          data[1]: test[1][i], 
                                                          data[2]: 1}))
    acc = np.mean(acc)
    if iteration % show_step == 0:
        print("step: {} accuracy: {:.4f}".format(iteration, acc))
    return acc
    
def save_summaries(sess, data, train_summary, test_summary, writer, batch, acc, iteration):
        s = train_summary.eval(session=sess, feed_dict={data[0]: batch[0],
                                                        data[1]: batch[1],
                                                        data[2]: 1})
        writer.add_summary(s, iteration)
        s = test_summary.eval(session=sess, feed_dict={data[4]: acc})
        writer.add_summary(s, iteration)
        writer.flush()
        
def run(scope, rate=1, conv=None):
    
    print("Running model: {}".format(scope))
    
    x = tf.placeholder(tf.float32, [None, 784])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    labels = tf.placeholder(tf.float32, [None, 10])
    
    scalar = tf.placeholder(tf.float32) # for summary
    
    data = [x, labels, keep_prob, learning_rate, scalar]

    if conv is None:
        net = Lenet(x, keep_prob, device='/gpu:0', rate=rate)
    else:
        net = Lenet(x, keep_prob, device='/gpu:0', conv=conv, rate=rate)
        
    predict = net.y

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=labels))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(predict,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

    with tf.name_scope(scope):
        with tf.name_scope("test"):
            test_summary = tf.summary.scalar('accuracy', scalar)

        with tf.name_scope("train"):
            train1 = tf.summary.scalar('accuracy', accuracy)
            train2 = tf.summary.scalar('loss', cross_entropy)
            train_summary = tf.summary.merge([train1, train2])

    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.as_default()
    sess.run(tf.global_variables_initializer())
    
    for iteration in range(iterations):
        batch = mnist.train.next_batch(batch_size)
        if iteration % display_step == 0:
            acc = show_acc(sess, data, accuracy, iteration)
            save_summaries(sess, data, train_summary, test_summary, writer, batch, acc, iteration)
        train(sess, data, train_step, batch, 0.5, lr)

    acc = show_acc(sess, data, accuracy, iterations)
    save_summaries(sess, data, train_summary, test_summary, writer, batch, acc, iterations)

    sess.close()


In [3]:
rates = [1, 1.4, 1.6, 1.8, 2] # feature map scale

run('basic')
for rate in rates:
    run("bin_weights/vector/{}".format(rate), rate, conv=c.conv_bin_weights_vector)
for rate in rates:
    run("bin_weights/scalar/{}".format(rate), rate, conv=c.conv_bin_weights_scalar)
for rate in rates:
    run("bin_activ/{}".format(rate), rate, conv=c.conv_bin_activ)

Running model: basic
step: 0 accuracy: 0.1164
step: 1000 accuracy: 0.9594
step: 2000 accuracy: 0.9743
step: 3000 accuracy: 0.9792
step: 4000 accuracy: 0.9838
Running model: bin_weights/vector/1
step: 0 accuracy: 0.0959
step: 1000 accuracy: 0.9482
step: 2000 accuracy: 0.9675
step: 3000 accuracy: 0.9749
step: 4000 accuracy: 0.9803
Running model: bin_weights/vector/1.4
step: 0 accuracy: 0.0996
step: 1000 accuracy: 0.9578
step: 2000 accuracy: 0.9732
step: 3000 accuracy: 0.9784
step: 4000 accuracy: 0.9801
Running model: bin_weights/vector/1.6
step: 0 accuracy: 0.1184
step: 1000 accuracy: 0.9597
step: 2000 accuracy: 0.9752
step: 3000 accuracy: 0.9785
step: 4000 accuracy: 0.9839
Running model: bin_weights/vector/1.8
step: 0 accuracy: 0.1060
step: 1000 accuracy: 0.9643
step: 2000 accuracy: 0.9758
step: 3000 accuracy: 0.9806
step: 4000 accuracy: 0.9842
Running model: bin_weights/vector/2
step: 0 accuracy: 0.1217
step: 1000 accuracy: 0.9616
step: 2000 accuracy: 0.9735
step: 3000 accuracy: 0.9810